In [23]:
import json
import os
from openai import AzureOpenAI

config_path = 'config.json'
with open(config_path, 'r') as config_file:
    config = json.load(config_file)

azure_api_key = config['azure_api_key']
azure_api_version = config['azure_api_version']
azure_endpoint = config['azure_endpoint']
deployment_name = config['deployment_name']

client = AzureOpenAI(
    api_key=azure_api_key,  
    api_version=azure_api_version,
    azure_endpoint=azure_endpoint
)


def generate_text(prompt):

    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "You are a math tutor."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=200,
        temperature=0.7
    )

    text = response.choices[0].message.content.strip()

    return text

def generate_question_answer_solution(grade, difficulty, topic):
    prompt = f"Generate a {difficulty} level practice question for grade {grade} on the topic of {topic}. Provide the question with heading Q, the answer as just the number with heading A, and a very short solution that is within 500 characters with heading S. Do not add other headings and do not use LaTeX or special formatting"

    text = generate_text(prompt)
    
    parts = text.split("\n\n")

    question = ""
    answer = ""
    solution = ""

    for part in parts:
        if part.startswith("Q:"):
            question = part.replace("Q:", "").strip()
        elif part.startswith("A:"):
            answer = part.replace("A:", "").strip()
        elif part.startswith("S:"):
            solution = part.replace("S:", "").strip()

    return question, answer, solution

def generate_solution(question):
    prompt = f"Generate a solution for the question {question}, do not give anything else or give any heading and do not use LaTeX or special formatting"

    text = generate_text(prompt)
    
    return text

def generate_quiz(grade, topic, number):
    if number < 3:
        number = 3
    elif number > 10:
        number = 10

    prompt_e = f"Generate {number} diffrent easy level practice questions for grade {grade} on the topic of {topic}. Provide the question with heading Q:, the answer as just the number with heading A:. Do not add other headings and do not use LaTeX or special formatting"

    text_e = generate_text(prompt_e)
    questions_e, answers_e = parse_questions_answers(text_e)
    
    prompt_m = f"Generate {number-1} diffrent medium level practice questions for grade {grade} on the topic of {topic}. Provide the question with heading Q:, the answer as just the number with heading A:. Do not add other headings and do not use LaTeX or special formatting"

    text_m = generate_text(prompt_m)
    questions_m, answers_m = parse_questions_answers(text_m)
    
    prompt_h = f"Generate {number-2} diffrent hard level practice questions for grade {grade} on the topic of {topic}. Provide the question with heading Q:, the answer as just the number with heading A:. Do not add other headings and do not use LaTeX or special formatting"

    text_h = generate_text(prompt_h)
    questions_h, answers_h = parse_questions_answers(text_h)

    return (questions_e, answers_e), (questions_m, answers_m), (questions_h, answers_h)

def check_answer(choice,answer):
    if choice == answer:
        return True
    else:
        return False

def parse_questions_answers(text):
    questions = []
    answers = []
    
    parts = text.split('\n')
    for part in parts:
        if part.startswith('Q'):
            questions.append(part.split('Q:', 1)[1].strip())
        elif part.startswith('A'):
            answers.append(part.split('A:', 1)[1].strip())
    
    return questions, answers





In [24]:
# Examples
grade = 5
difficulty = "medium"
topic = "ratio"
question, answer, solution = generate_question_answer_solution(grade, difficulty, topic)
text = generate_question_answer_solution(grade, difficulty, topic)
print("Question:", question)
print("Answer:", answer)
print("Solution:", solution)


Question: In a school cafeteria, there are 60 apples and 45 oranges. What is the simplest form of the ratio of apples to oranges?
Answer: 4:3
Solution: To simplify the ratio, find the greatest common divisor (GCD) of 60 and 45. The GCD of 60 and 45 is 15. Divide both numbers by 15: 60 ÷ 15 = 4 and 45 ÷ 15 = 3. So, the simplest form of the ratio is 4:3.


In [25]:
grade = 1
topic = "addition"
number = 4

output = generate_quiz(grade, topic, number)

print("Easy Questions and Answers:", output[0])
print("Medium Questions and Answers:", output[1])
print("Hard Questions and Answers:", output[2])

Easy Questions and Answers: (['2 + 3', '1 + 4', '5 + 2', '3 + 3'], ['5', '5', '7', '6'])
Medium Questions and Answers: (['What is 7 + 5?', 'What is 8 + 6?', 'What is 9 + 4?'], ['12', '14', '13'])
Hard Questions and Answers: (['What is 17 + 8?', 'What is 14 + 9?'], ['25', '23'])
